In [1]:
import tensorflow as tf
import pandas as pd
from keras import Sequential
from keras import layers
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import os
os.getcwd()

'/content'

In [0]:
os.chdir('/content/drive/My Drive/Santander_Kaggle')

In [5]:
!ls

l3bs10drop10.csv	submission_nn.csv
l3bs20.csv		test.csv
sample_submission.csv	train.csv
santander_kaggle.ipynb	units_200_hidden_1_regularized_drop10.csv


In [0]:
train = pd.read_csv('train.csv')

In [0]:
test = pd.read_csv('test.csv')

In [8]:
#Check num of cases in label 
print(train.target.value_counts())
print(train.target.value_counts()[1]/train.target.value_counts()[0])

0    179902
1     20098
Name: target, dtype: int64
0.1117163789174106


In [0]:
train_df, test_df = train_test_split(train, test_size = 0.25, random_state = 1984)

In [10]:
#Test - Make sure labels are equally distributed in train and test set
train_df.target.sum()/train_df.shape[0]

0.10054666666666667

In [11]:
test_df.target.sum()/test_df.shape[0]

0.10032

In [0]:
train_features = train.drop(['target', 'ID_code'], axis=1)
train_targets = train['target']
test_features = test.drop(['ID_code'], axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets, test_size = 0.25, random_state = 2019)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
test_features = sc.transform(test_features)

In [0]:
# Add RUC metric to monitor NN
from keras import backend as K

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
# Build ANN
from keras.layers.core import Dense
from keras import regularizers
from keras.layers import Dropout

In [0]:
from keras.constraints import max_norm

In [61]:
input_dim = X_train.shape[1]
input_dim

200

In [0]:
model = Sequential()

In [94]:
# Input layer
#model.add(Dense(units = 200, activation = "relu", input_dim = input_dim, kernel_initializer = "uniform", kernel_regularizer=regularizers.l2(0.005)))
model.add(Dense(units = 200, activation = "relu", input_dim = input_dim, kernel_initializer = "normal", kernel_regularizer=regularizers.l2(0.005), 
                kernel_constraint = max_norm(5.)))
# Add dropout regularization
model.add(Dropout(rate=0.2))

# First hidden layer
model.add(Dense(units = 200, activation='relu', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=max_norm(5)))
# Add dropout regularization
model.add(Dropout(rate=0.2))

#input_dim=input_dim

# Second hidden layer
model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=max_norm(5)))
# Add dropout regularization
model.add(Dropout(rate=0.1))

# Third hidden layer
model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=max_norm(5)))
# Add dropout regularization
#model.add(Dropout(rate=0.1))

# Fourth hidden layer
#model.add(Dense(25, activation='tanh', kernel_regularizer=regularizers.l2(0.005)))
# Add dropout regularization
#model.add(Dropout(rate=0.1))

# Output layer
model.add(layers.Dense(units = 1, activation='sigmoid'))

#model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_63 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_64 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_65 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_84 (Dense)             (None, 50)                5050      
__________

In [95]:
#batch_size = 20,
model.fit(X_train, y_train, batch_size = 10, epochs = 4, validation_data = (X_test, y_test))

Train on 150000 samples, validate on 50000 samples
Epoch 1/4
150000/150000 [==============================] - 174s 1ms/step - loss: 0.3283 - acc: 0.9034 - auc: 0.7962 - val_loss: 0.2847 - val_acc: 0.9086 - val_auc: 0.8146
Epoch 2/4
150000/150000 [==============================] - 149s 991us/step - loss: 0.2833 - acc: 0.9048 - auc: 0.8163 - val_loss: 0.2733 - val_acc: 0.9101 - val_auc: 0.8183
Epoch 3/4
150000/150000 [==============================] - 148s 988us/step - loss: 0.2806 - acc: 0.9048 - auc: 0.8197 - val_loss: 0.2744 - val_acc: 0.9081 - val_auc: 0.8204
Epoch 4/4
150000/150000 [==============================] - 149s 994us/step - loss: 0.2795 - acc: 0.9045 - auc: 0.8208 - val_loss: 0.2805 - val_acc: 0.9094 - val_auc: 0.8205


In [0]:
# Compute roc
from sklearn.metrics import roc_auc_score

In [96]:
y_pred = model.predict_proba(X_test)
roc_auc_score(y_test, y_pred)

0.8392294351630342

In [0]:
def predict_and_save(model, filename):
  id_code_test = test['ID_code']
  # Make predicitions
  pred = model.predict(test_features)
  pred_ = pred[:,0]
  fn = filename + ".csv"
  # To CSV
  foo = pd.DataFrame({"ID_code" : id_code_test, "target" : pred_}).to_csv(fn, index = False, header = True)

In [0]:
predict_and_save(model, "units_200_hidden_1_regularized_drop10")